<a href="https://colab.research.google.com/github/aniruddh47/AI_DOCTOR/blob/main/AI_DOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# 1. Load & clean data
df = pd.read_csv("/content/Disease and symptoms dataset.csv")
df.columns = df.columns.str.strip().str.lower()
df.fillna(0, inplace=True)

# 2. Separate features and target
X = df.drop("diseases", axis=1)
y = df["diseases"]

# 3. Encode target (diseases)
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# 4. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 5. Build neural network
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y_encoded)), activation='softmax')  # Output layer for multi-class
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 6. Train with epochs & early stopping
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.1,
    callbacks=[EarlyStopping(patience=3, restore_best_weights=True)],
    verbose=1
)

# 7. Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {accuracy*100:.2f}%")

# 8. Save model and encoder
model.save("neural_disease_model.h5")
import pickle
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
5557/5557 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.5798 - loss: 2.1069 - val_accuracy: 0.8346 - val_loss: 0.4872
Epoch 2/30
5557/5557 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.8405 - loss: 0.4538 - val_accuracy: 0.8428 - val_loss: 0.4369
Epoch 3/30
5557/5557 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.8506 - loss: 0.4072 - val_accuracy: 0.8483 - val_loss: 0.4197
Epoch 4/30
5557/5557 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.8564 - loss: 0.3843 - val_accuracy: 0.8457 - val_loss: 0.4141
Epoch 5/30
5557/5557 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.8571 - loss: 0.3742 - val_accuracy: 0.8498 - val_loss: 0.4117
Epoch 6/30
5557/5557 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.8608 - loss: 0.3634 - val_accuracy: 0.8504 - val_loss: 0.3992
Epoch 7/30
5557/5557 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.8647 - loss: 0.3508 - val_accuracy: 0.8529 - val_loss: 0.3919
Epoch 8/30
5557/5557 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.8647 - loss: 0

✅ Test Accuracy: 85.31%


In [2]:
model.save("disease_model.keras")

In [3]:
import pickle

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)


In [4]:
model.save("disease_model.keras")
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)


In [5]:
import numpy as np
from tensorflow.keras.models import load_model
import pickle

# === Load Trained Model ===
model = load_model("disease_model.keras")

# === Load Label Encoder ===
with open("label_encoder.pkl", "rb") as f:
    encoder = pickle.load(f)

# === Load or Create Input Symptom Vector ===
# Replace this vector with actual symptoms in the same format as your training data
# 377 binary values: 1 = symptom present, 0 = not
# Creating a sample input vector from the first row of the training data X
input_vector = X.iloc[0].values.reshape(1, -1) # Assuming X is available from previous cells

# === Make Prediction ===
prediction = model.predict(input_vector)
predicted_label_encoded = np.argmax(prediction, axis=1)
predicted_label = encoder.inverse_transform(predicted_label_encoded)

print("🩺 Predicted Disease:", predicted_label[0])

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
🩺 Predicted Disease: panic disorder


In [6]:
import numpy as np

# Example vector for testing — replace with real symptoms
symptom_vector = np.zeros(377)
symptom_vector[0] = 1  # e.g. anxiety and nervousness
symptom_vector[2] = 1  # e.g. shortness of breath
symptom_vector[6] = 1  # e.g. insomnia

np.save("input_vector.npy", symptom_vector)


In [7]:
import numpy as np
from tensorflow.keras.models import load_model
import pickle

# Load model
model = load_model("/content/disease_model.keras")

# Load encoder
with open("/content/label_encoder.pkl", "rb") as f:
    encoder = pickle.load(f)

# Agar input_vector.npy upload kiya hai:
input_vector = np.load("input_vector.npy")
input_vector = input_vector.reshape(1, -1)

# Predict
pred = model.predict(input_vector)
predicted_label = encoder.inverse_transform([np.argmax(pred)])

print("🩺 Predicted Disease:", predicted_label[0])


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
🩺 Predicted Disease: acute stress reaction


In [9]:
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import load_model

# === Load Model and Encoder ===
model = load_model("/content/disease_model.keras")
with open("/content/label_encoder.pkl", "rb") as f:
    encoder = pickle.load(f)

# === Load Dataset to Get Symptom Columns ===
df = pd.read_csv("/content/Disease and symptoms dataset.csv")
df.columns = df.columns.str.strip().str.lower()
symptom_columns = df.drop("diseases", axis=1).columns.tolist()

# === Get Paragraph from User ===
print("📝 Please describe how you're feeling (in one or two sentences):")
paragraph = input("Your description: ").lower()

# === Extract Matching Symptoms ===
found_symptoms = [symptom for symptom in symptom_columns if symptom in paragraph]
print("🔍 Detected Symptoms:", found_symptoms)

# === Create Binary Symptom Vector ===
symptom_vector = np.zeros(len(symptom_columns))
for i, symptom in enumerate(symptom_columns):
    if symptom in found_symptoms:
        symptom_vector[i] = 1

# === Predict Disease ===
input_vector = symptom_vector.reshape(1, -1)
pred = model.predict(input_vector)
predicted_label = encoder.inverse_transform([np.argmax(pred)])

print("🩺 Predicted Disease:", predicted_label[0])



/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


📝 Please describe how you're feeling (in one or two sentences):
Your description: Your description: I have been coughing a lot and have chest pain with shortness of breath.
🔍 Detected Symptoms: ['shortness of breath', 'cough']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step
🩺 Predicted Disease: tietze syndrome
